<a href="https://colab.research.google.com/github/supremmm/GuardClauses/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

# 1- Exploratory Data Analysis

Before modeling, we explore the data to understand relationships between variables

In [ ]:
# Pairplot shows relationships between all numerical variables, colored by 'smoker'
sns.pairplot(dataset, hue='smoker', palette='viridis', corner=True)
plt.suptitle('Pairplot of numerical features by smoker status', y=1.02)
plt.show()

**seaborn.pairplot:** This function creates a grid of plots showing the relationship between each pair of numerical variables.
By setting **hue='smoker'**, we can instantly see that smokers (orange dots) consistently have higher expenses than non-smokers (purple dots), especially as age and BMI increase.

In [ ]:
# A more focused look at key relationships
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.scatterplot(data=dataset, x='bmi', y='expenses', hue='smoker', alpha=0.6, ax=axes[0], palette='magma')
axes[0].set_title('Expenses vs. BMI by smoker status')
sns.scatterplot(data=dataset, x='age', y='expenses', hue='smoker', alpha=0.6, ax=axes[1], palette='magma')
axes[1].set_title('Expenses vs. Age by smoker status')
plt.tight_layout()
plt.show()

**seaborn.scatterplot:** We create focused scatter plots to examine the expenses vs. bmi and expenses vs. age relationships. These plots confirm that expenses rise with age and that smokers form a distinct, higher-cost group compared to non-smokers.

# 2- data processing

In [ ]:



print("Preprocessing data...")
# Create a copy to avoid modifying the original dataframe during preprocessing
df_processed = dataset.copy()

# Feature Engineering: Create interaction terms
# This is a good way to capture complex relationships
df_processed['bmi_smoker'] = df_processed['bmi'] * (df_processed['smoker'] == 'yes').astype(int)

# Convert categorical columns to numerical using one-hot encoding
df_processed = pd.get_dummies(df_processed, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=float)

# Separate features (X) and target (y)
X = df_processed.drop('expenses', axis=1)
y = df_processed['expenses']

# Split data into training and testing sets using scikit-learn
# This is more robust and standard than manual sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = X_train
train_labels = y_train

test_dataset = X_test
test_labels = y_test

print("Preprocessing completed.")
print("Training dataset shape:", train_dataset.shape)
print("Testing dataset shape:", test_dataset.shape)


**Normalization:** A layers.Normalization layer is created and adapted only to the training data. This scales all features to a similar range, which helps the neural network train faster and more reliably.

In [ ]:
# Normalize numerical features using a Keras layer
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train)) # Adapt the normalizer only on the training data


# 3- build the model

**Architecture**: The model consists of the normalization layer followed by two hidden Dense layers (with 128 and 64 neurons, respectively) using the 'ReLU' activation function, and a final Dense layer with a single neuron to output the predicted expense.

**Compiler:** The model is compiled with the Adam optimizer and uses Mean Absolute Error (MAE) as its primary loss function, as it is easily interpretable in terms of dollar error.

In [ ]:

def build_model(normalizer_layer):
    model = keras.Sequential([
        normalizer_layer,
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_absolute_error',
        metrics=['mae', 'mse']
    )
    return model

model = build_model(normalizer)
model.summary()

# training the model

The model is trained using the .fit() method on the training data .

**Early Stopping:** We use an EarlyStopping callback to monitor the validation loss. If the loss on the validation set does not improve for a set number of epochs (patience=15), training is stopped automatically. This prevents overfitting and saves computational time. The restore_best_weights=True argument ensures the model reverts to the state with the lowest validation error.

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(
    train_dataset,
    train_labels,
    epochs=500,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1)
print("Training finished.")

# 5- Evaluation

After training, we evaluate the model's performance.

**History Plotting:** The plot_history function visualizes the training and validation MAE and MSE over epochs. This helps diagnose if the model was overfitting or underfitting.

**Test Set Evaluation:** The model's final performance is measured on the unseen test set. This gives us a reliable estimate of how the model would perform on new data.

In [ ]:
def plot_history(history_obj):
    hist = pd.DataFrame(history_obj.history)
    hist['epoch'] = history_obj.epoch

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Absolute Error (Loss)')
    plt.plot(hist['epoch'], hist['mae'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
    plt.legend()
    plt.title('Training and Validation MAE')


    plt.subplot(1, 2, 2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.plot(hist['epoch'], hist['mse'], label='Train MSE')
    plt.plot(hist['epoch'], hist['val_mse'], label='Val MSE')
    plt.legend()
    plt.title('Training and Validation MSE')

    plt.tight_layout()
    plt.show()

plot_history(history)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


**Predictions vs. True Values Plot:** This scatter plot compares the model's predicted expenses against the actual expenses from the test set. A perfect model would have all points lying on the 45-degree diagonal line. This visualization provides an intuitive check of the model's accuracy across the range of values.